### Ex1.a)

In [ ]:
from pgmpy.models import MarkovNetwork
from pgmpy.factors.discrete import DiscreteFactor
from pgmpy.inference import BeliefPropagation
import networkx as nx
import numpy as np

model = MarkovNetwork([('A1', 'A2'), ('A1', 'A3'), ('A2', 'A4'), 
                       ('A2', 'A5'), ('A3', 'A4'), ('A4', 'A5')])

pos = nx.spring_layout(model)
nx.draw(model, with_labels=True, pos=pos, node_size=2000)

cliques = list(nx.find_cliques(model))
print("Clicurile modelului:")
for clique in cliques:
    print(clique)

### Ex1.b)

In [ ]:
def create_factor(variables):
    cardinality = [2] * len(variables)
    n = 2 ** len(variables)
    values = np.zeros(n)
    
    for i in range(n):
        config = []
        temp = i
        for j in range(len(variables)):
            config.append(-1 if temp % 2 == 0 else 1)
            temp //= 2
        
        exponent = sum((idx+1) * val for idx, val in enumerate(config))
        values[i] = np.exp(exponent)
    
    return DiscreteFactor(variables=variables, cardinality=cardinality, values=values)

factors = []
for clique in cliques:
    factor = create_factor(clique)
    factors.append(factor)
    model.add_factors(factor)

print("\nModelul este valid:", model.check_model())

bp_infer = BeliefPropagation(model)
map_result = bp_infer.map_query(variables=['A1', 'A2', 'A3', 'A4', 'A5'])
print("\nConfiguratia optima:")
print(map_result)

### Ex2.a)

In [ ]:
np.random.seed(42)
original_image = np.random.choice([0, 1], size=(5, 5))
print("Imaginea originala:")
print(original_image)

noisy_image = original_image.copy()
num_noisy = int(0.1 * 25)
noisy_positions = np.random.choice(25, num_noisy, replace=False)
for pos in noisy_positions:
    i, j = pos // 5, pos % 5
    noisy_image[i, j] = 1 - noisy_image[i, j]

print("\nImaginea cu zgomot:")
print(noisy_image)

edges = []
for i in range(5):
    for j in range(5):
        node = f"P_{i}_{j}"
        if i < 4:  # Sud
            edges.append((node, f"P_{i+1}_{j}"))
        if j < 4:  # Est
            edges.append((node, f"P_{i}_{j+1}"))

mrf_model = MarkovNetwork(edges)

lambda_param = 1.0

def energy_to_prob(xi, xj, yi):
    data_term = lambda_param * (xi - yi) ** 2
    smooth_term = (xi - xj) ** 2 if xj is not None else 0
    energy = data_term + smooth_term
    return np.exp(-energy)

for i in range(5):
    for j in range(5):
        node = f"P_{i}_{j}"
        yi = noisy_image[i, j]
        
        values_unary = [energy_to_prob(0, None, yi), energy_to_prob(1, None, yi)]
        factor_unary = DiscreteFactor(variables=[node], cardinality=[2], values=values_unary)
        mrf_model.add_factors(factor_unary)
        
        if i < 4:
            neighbor = f"P_{i+1}_{j}"
            values_pair = [energy_to_prob(0, 0, yi), energy_to_prob(0, 1, yi),
                          energy_to_prob(1, 0, yi), energy_to_prob(1, 1, yi)]
            factor_pair = DiscreteFactor(variables=[node, neighbor], 
                                        cardinality=[2, 2], values=values_pair)
            mrf_model.add_factors(factor_pair)
        
        if j < 4:
            neighbor = f"P_{i}_{j+1}"
            values_pair = [energy_to_prob(0, 0, yi), energy_to_prob(0, 1, yi),
                          energy_to_prob(1, 0, yi), energy_to_prob(1, 1, yi)]
            factor_pair = DiscreteFactor(variables=[node, neighbor], 
                                        cardinality=[2, 2], values=values_pair)
            mrf_model.add_factors(factor_pair)

### Ex2.b)

In [ ]:
bp_mrf = BeliefPropagation(mrf_model)
variables = [f"P_{i}_{j}" for i in range(5) for j in range(5)]
map_result = bp_mrf.map_query(variables=variables)

denoised_image = np.zeros((5, 5), dtype=int)
for i in range(5):
    for j in range(5):
        node = f"P_{i}_{j}"
        denoised_image[i, j] = map_result[node]

print("\nImaginea reconstruita:")
print(denoised_image)

print("\nComparatie:")
print(f"Diferente original vs zgomot: {np.sum(original_image != noisy_image)}")
print(f"Diferente original vs denoised: {np.sum(original_image != denoised_image)}")